In [4]:
import cv2
import time

# 웹캠 캡처 객체 생성
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    exit()


# 카운트다운 시간 설정
countdown_numbers = ["", "", 3, 2, 1, "Start"]

for count in countdown_numbers:
    # 1초 동안 텍스트를 표시
    start_time = time.time()
    while time.time() - start_time <= 1:
        ret, frame = cap.read()

        if not ret:
            break

        # 화면 크기
        height, width, _ = frame.shape

        # 텍스트 설정
        text = str(count)
        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = 2
        font_thickness = 5
        color = (255, 255, 255)

        # 텍스트 중앙 배치
        text_size = cv2.getTextSize(text, font, font_scale, font_thickness)[0]
        text_x = (width - text_size[0]) // 2
        text_y = (height + text_size[1]) // 2

        # 텍스트 추가
        cv2.putText(frame, text, (text_x, text_y), font, font_scale, color, font_thickness)

        # 화면 표시
        cv2.imshow("Camera", frame)

        # 'q'를 누르면 종료
        if cv2.waitKey(1) == ord('q'):
            cap.release()
            cv2.destroyAllWindows()
            exit()

# 텍스트 사라진 화면 유지
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # 화면 표시 (텍스트 없음)
    cv2.imshow("Camera", frame)

    # 'q'를 누르면 종료
    if cv2.waitKey(1) == ord('q'):
        break

# 자원 해제
cap.release()
cv2.destroyAllWindows()